In [53]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import random
import torchvision.datasets as dsets
import torchvision.transforms as transforms #should be used to convert images to pytorch tensors
from matplotlib.pyplot import imshow

In [55]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

In [56]:
training_epoch = 15
batch_size = 100
learning_rate = 0.001

In [57]:
cifar10_train = dsets.CIFAR10(root='/content/drive/MyDrive/CSCE464/datasets/CIFAR10',
                          train= True,
                          transform= transforms.ToTensor(),
                          download= True)

cifar10_test = dsets.CIFAR10(root='/content/drive/MyDrive/CSCE464/datasets/CIFAR10',
                          train= False,
                          transform= transforms.ToTensor(),
                          download= True)

Files already downloaded and verified
Files already downloaded and verified


In [58]:
cifar10_train

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /content/drive/MyDrive/CSCE464/datasets/CIFAR10
    Split: Train
    StandardTransform
Transform: ToTensor()

In [59]:
cifar10_test

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: /content/drive/MyDrive/CSCE464/datasets/CIFAR10
    Split: Test
    StandardTransform
Transform: ToTensor()

In [60]:
data_loader = torch.utils.data.DataLoader(dataset= cifar10_train,
                                          batch_size= batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [61]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.layer3 = torch.nn.Sequential(
        torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.fc = torch.nn.Linear(2048, 10, bias=True)
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1) #reduce from 2d data to 1d data
    out = self.fc(out)

    return out

In [62]:
model = CNN().to(device)

In [63]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [64]:
total_batch = len(data_loader)

for epoch in range(training_epoch):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    pred = model(X).to(device)
    cost = loss(pred, Y).to(device)
    correct_pred = torch.argmax(pred, axis=1) == Y
    accuracy = correct_pred.float().mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print("Epoch {:2d} / {} Cost: {:.5f} Training ACC: {:.2f}%".format(epoch+1, 15, avg_cost, accuracy * 100))

Epoch  1 / 15 Cost: 1.53364 Training ACC: 54.00%
Epoch  2 / 15 Cost: 1.14015 Training ACC: 63.00%
Epoch  3 / 15 Cost: 0.98177 Training ACC: 71.00%
Epoch  4 / 15 Cost: 0.88119 Training ACC: 64.00%
Epoch  5 / 15 Cost: 0.79993 Training ACC: 69.00%
Epoch  6 / 15 Cost: 0.74435 Training ACC: 76.00%
Epoch  7 / 15 Cost: 0.69378 Training ACC: 73.00%
Epoch  8 / 15 Cost: 0.64343 Training ACC: 82.00%
Epoch  9 / 15 Cost: 0.60733 Training ACC: 81.00%
Epoch 10 / 15 Cost: 0.57247 Training ACC: 75.00%
Epoch 11 / 15 Cost: 0.53367 Training ACC: 85.00%
Epoch 12 / 15 Cost: 0.49939 Training ACC: 79.00%
Epoch 13 / 15 Cost: 0.47365 Training ACC: 82.00%
Epoch 14 / 15 Cost: 0.44300 Training ACC: 80.00%
Epoch 15 / 15 Cost: 0.40875 Training ACC: 84.00%


In [65]:
test_loader = torch.utils.data.DataLoader(dataset= cifar10_test,
                                          batch_size= len(cifar10_test),
                                          shuffle = True,
                                          drop_last = True)

In [66]:
model.eval()
with torch.no_grad():
  for data1, data2 in test_loader:

    x_test = data1.to(device)
    y_test =  data2.to(device)
    prediction = model(x_test).to(device)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()

  print('Testing Accuracy: {:.2f}%'.format(accuracy.item()*100))

Testing Accuracy: 73.26%
